In [1]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [2]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [3]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [4]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [5]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [6]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [7]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [8]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '947688', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '283412', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '775051', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [9]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [10]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [11]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [12]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [13]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",May-21,"25,580","24,993",587,2.3,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",May-21,"99,524","97,175","2,349",2.4,01003
38629,CN0100500000000,1,5,"Barbour County, AL",May-21,"8,103","7,732",371,4.6,01005
38630,CN0100700000000,1,7,"Bibb County, AL",May-21,"8,555","8,293",262,3.1,01007
38631,CN0100900000000,1,9,"Blount County, AL",May-21,"24,709","24,238",471,1.9,01009


In [14]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [15]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",May-21,"13,220","11,808",1412,10.7,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",May-21,"2,637","2,316",321,12.2,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",May-21,"7,079","6,267",812,11.5,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",May-21,"8,471","7,606",865,10.2,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",May-21,"9,433","8,309",1124,11.9,72153


In [16]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [17]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [18]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [19]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [20]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [21]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [22]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [23]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [24]:
covid.tail(100)

,date,county,state,fips,cases,deaths
1582577,2021-08-03,Webster,West Virginia,54101.0,616,9.0
1582578,2021-08-03,Wetzel,West Virginia,54103.0,1435,32.0
1582579,2021-08-03,Wirt,West Virginia,54105.0,474,4.0
1582580,2021-08-03,Wood,West Virginia,54107.0,8095,177.0
1582581,2021-08-03,Wyoming,West Virginia,54109.0,2124,41.0
1582582,2021-08-03,Adams,Wisconsin,55001.0,2002,21.0
1582583,2021-08-03,Ashland,Wisconsin,55003.0,1420,15.0
1582584,2021-08-03,Barron,Wisconsin,55005.0,6100,86.0
1582585,2021-08-03,Bayfield,Wisconsin,55007.0,1297,21.0
1582586,2021-08-03,Brown,Wisconsin,55009.0,36133,310.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [25]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_PROBABLE_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_PROBABLE_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_PROBABLE_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_PROBABLE_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_PROBABLE_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
515,07/28/2021,1151,467,54,6,0,892,1244,45,3,4,174,94,10,2,0,116,61,177,9,0,0,361,150,12,1,0,303,109,413,13,0,1,238,78,10,2,0,181,72,253,6,1,1,272,118,20,1,0,209,83,292,12,1,1,106,27,3,0,0,83,26,109,4,1,1,11000
516,07/29/2021,1180,430,50,6,1,932,1298,46,4,4,157,66,9,0,0,120,64,184,9,0,0,416,131,17,1,0,319,115,434,14,1,1,214,84,8,1,0,183,73,256,7,1,1,284,105,9,4,0,223,86,310,13,1,1,109,43,7,0,0,86,27,114,5,1,1,11000
517,07/30/2021,1126,442,49,5,2,971,1357,48,4,5,137,71,14,1,0,123,68,190,10,0,0,379,137,10,1,0,331,120,451,13,1,1,218,84,10,2,0,189,75,264,8,1,1,283,107,14,1,0,235,91,327,13,1,1,109,41,1,0,0,94,31,124,4,1,1,11000
518,07/31/2021,691,358,46,3,1,997,1397,48,4,5,106,68,8,0,0,130,70,201,10,0,0,217,97,17,2,0,337,122,458,14,1,1,115,69,5,1,0,192,78,270,7,1,1,168,107,12,0,0,240,99,339,13,1,1,85,15,5,0,0,98,30,128,4,1,1,11000
519,08/01/2021,631,345,23,3,4,1004,1409,47,4,5,71,61,7,0,0,132,72,204,10,0,0,241,90,4,1,0,342,121,463,13,1,1,103,65,5,1,0,193,77,270,7,1,1,153,98,5,1,0,240,103,343,12,1,1,63,26,2,0,0,98,30,128,4,0,0,11000


In [26]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [27]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [28]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [29]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [30]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [31]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [32]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
515,07/28/2021,174,2,New York,Bronx,36005.0
516,07/29/2021,157,0,New York,Bronx,36005.0
517,07/30/2021,137,1,New York,Bronx,36005.0
518,07/31/2021,106,0,New York,Bronx,36005.0
519,08/01/2021,71,0,New York,Bronx,36005.0


In [33]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [34]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [35]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [36]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
515,07/28/2021,361,1,Kings,New York,36047.0
516,07/29/2021,416,1,Kings,New York,36047.0
517,07/30/2021,379,1,Kings,New York,36047.0
518,07/31/2021,217,2,Kings,New York,36047.0
519,08/01/2021,241,1,Kings,New York,36047.0


In [37]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [39]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [40]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [41]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,3654,8


In [42]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [43]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [44]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
506,07/19/2021,168,1,New York,Queens,36081.0,2021-07-19
507,07/20/2021,142,1,New York,Queens,36081.0,2021-07-20
508,07/21/2021,190,0,New York,Queens,36081.0,2021-07-21
509,07/22/2021,185,1,New York,Queens,36081.0,2021-07-22
510,07/23/2021,201,1,New York,Queens,36081.0,2021-07-23
511,07/24/2021,135,0,New York,Queens,36081.0,2021-07-24
512,07/25/2021,153,1,New York,Queens,36081.0,2021-07-25
513,07/26/2021,252,0,New York,Queens,36081.0,2021-07-26
514,07/27/2021,267,1,New York,Queens,36081.0,2021-07-27
515,07/28/2021,272,1,New York,Queens,36081.0,2021-07-28


In [45]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
470,06/13/2021,Richmond,New York,36085.0,12,0.0,2021-06-13
471,06/14/2021,Richmond,New York,36085.0,11,1.0,2021-06-14
472,06/15/2021,Richmond,New York,36085.0,18,1.0,2021-06-15
473,06/16/2021,Richmond,New York,36085.0,24,0.0,2021-06-16
474,06/17/2021,Richmond,New York,36085.0,22,0.0,2021-06-17
475,06/18/2021,Richmond,New York,36085.0,20,2.0,2021-06-18
476,06/19/2021,Richmond,New York,36085.0,14,2.0,2021-06-19
477,06/20/2021,Richmond,New York,36085.0,9,0.0,2021-06-20
478,06/21/2021,Richmond,New York,36085.0,25,0.0,2021-06-21
479,06/22/2021,Richmond,New York,36085.0,17,0.0,2021-06-22


In [46]:
print (data_date_dt)

2021-08-04 00:00:00


In [47]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
1579428,1001.0,114.0
1579429,1003.0,329.0


In [48]:
Kings_death.head(2)

,fips,deaths
0,36047.0,8687


In [49]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
1582432,51690.0,79.0
1582433,51115.0,14.0
1582434,51117.0,68.0
1582435,51119.0,22.0
1582436,51121.0,81.0
1582437,51125.0,13.0
1582438,51127.0,19.0
1582439,51700.0,242.0
1582440,51710.0,274.0
1582441,51131.0,37.0


In [50]:
covid_death.shape

(3254, 2)

In [51]:
print (County_covid_death)

{1001.0: 114.0, 1003.0: 329.0, 1005.0: 61.0, 1007.0: 65.0, 1009.0: 139.0, 1011.0: 42.0, 1013.0: 72.0, 1015.0: 334.0, 1017.0: 125.0, 1019.0: 48.0, 1021.0: 117.0, 1023.0: 25.0, 1025.0: 63.0, 1027.0: 59.0, 1029.0: 45.0, 1031.0: 132.0, 1033.0: 142.0, 1035.0: 32.0, 1037.0: 29.0, 1039.0: 125.0, 1041.0: 58.0, 1043.0: 206.0, 1045.0: 117.0, 1047.0: 163.0, 1049.0: 192.0, 1051.0: 219.0, 1053.0: 83.0, 1055.0: 370.0, 1057.0: 65.0, 1059.0: 81.0, 1061.0: 83.0, 1063.0: 37.0, 1065.0: 78.0, 1067.0: 45.0, 1069.0: 293.0, 1071.0: 117.0, 1073.0: 1591.0, 1075.0: 38.0, 1077.0: 254.0, 1079.0: 100.0, 1081.0: 181.0, 1083.0: 158.0, 1085.0: 55.0, 1087.0: 52.0, 1089.0: 533.0, 1091.0: 67.0, 1093.0: 107.0, 1095.0: 236.0, 1097.0: 865.0, 1099.0: 41.0, 1101.0: 628.0, 1103.0: 291.0, 1105.0: 28.0, 1107.0: 62.0, 1109.0: 79.0, 1111.0: 44.0, 1113.0: 43.0, 1117.0: 259.0, 1115.0: 252.0, 1119.0: 33.0, 1121.0: 188.0, 1123.0: 156.0, 1125.0: 465.0, 1127.0: 288.0, 1129.0: 39.0, 1131.0: 31.0, 1133.0: 73.0, 2013.0: 3.0, 2016.0: 0.0, 

In [52]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [53]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [54]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
1537535,2021-07-21,New Haven,Connecticut,9009.0,92963,2134.0,2021-07-21,-14 days,NaN
1540781,2021-07-22,New Haven,Connecticut,9009.0,93029,2134.0,2021-07-22,-13 days,66.0
1544028,2021-07-23,New Haven,Connecticut,9009.0,93068,2134.0,2021-07-23,-12 days,39.0
1547274,2021-07-24,New Haven,Connecticut,9009.0,93068,2134.0,2021-07-24,-11 days,0.0
1550520,2021-07-25,New Haven,Connecticut,9009.0,93068,2134.0,2021-07-25,-10 days,0.0
1553766,2021-07-26,New Haven,Connecticut,9009.0,93208,2135.0,2021-07-26,-9 days,140.0
1557012,2021-07-27,New Haven,Connecticut,9009.0,93352,2135.0,2021-07-27,-8 days,144.0
1560258,2021-07-28,New Haven,Connecticut,9009.0,93463,2135.0,2021-07-28,-7 days,111.0
1563505,2021-07-29,New Haven,Connecticut,9009.0,93553,2140.0,2021-07-29,-6 days,90.0
1566752,2021-07-30,New Haven,Connecticut,9009.0,93680,2140.0,2021-07-30,-5 days,127.0


In [55]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [56]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,1239.0


In [57]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
470,06/13/2021,Richmond,New York,36085.0,12,0.0,2021-06-13
471,06/14/2021,Richmond,New York,36085.0,11,1.0,2021-06-14
472,06/15/2021,Richmond,New York,36085.0,18,1.0,2021-06-15
473,06/16/2021,Richmond,New York,36085.0,24,0.0,2021-06-16
474,06/17/2021,Richmond,New York,36085.0,22,0.0,2021-06-17
475,06/18/2021,Richmond,New York,36085.0,20,2.0,2021-06-18
476,06/19/2021,Richmond,New York,36085.0,14,2.0,2021-06-19
477,06/20/2021,Richmond,New York,36085.0,9,0.0,2021-06-20
478,06/21/2021,Richmond,New York,36085.0,25,0.0,2021-06-21
479,06/22/2021,Richmond,New York,36085.0,17,0.0,2021-06-22


In [58]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [59]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [60]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [61]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [62]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3219.0,72153.0,36.0,36439,72153,PUERTO RICO
3216,0.0,0.0,0.0,890,2060,ALASKA
3217,0.0,0.0,0.0,2132,2105,ALASKA
3218,0.0,0.0,0.0,689,2282,ALASKA
3219,0.0,0.0,0.0,1375,2164,ALASKA


In [63]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-08-05 01:25:34.676637
8


In [64]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [65]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [66]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [67]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [68]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 114.0
1003.0 329.0
1005.0 61.0
1007.0 65.0
1009.0 139.0
1011.0 42.0
1013.0 72.0
1015.0 334.0
1017.0 125.0
1019.0 48.0
1021.0 117.0
1023.0 25.0
1025.0 63.0
1027.0 59.0
1029.0 45.0
1031.0 132.0
1033.0 142.0
1035.0 32.0
1037.0 29.0
1039.0 125.0
1041.0 58.0
1043.0 206.0
1045.0 117.0
1047.0 163.0
1049.0 192.0
1051.0 219.0
1053.0 83.0
1055.0 370.0
1057.0 65.0
1059.0 81.0
1061.0 83.0
1063.0 37.0
1065.0 78.0
1067.0 45.0
1069.0 293.0
1071.0 117.0
1073.0 1591.0
1075.0 38.0
1077.0 254.0
1079.0 100.0
1081.0 181.0
1083.0 158.0
1085.0 55.0
1087.0 52.0
1089.0 533.0
1091.0 67.0
1093.0 107.0
1095.0 236.0
1097.0 865.0
1099.0 41.0
1101.0 628.0
1103.0 291.0
1105.0 28.0
1107.0 62.0
1109.0 79.0
1111.0 44.0
1113.0 43.0
1117.0 259.0
1115.0 252.0
1119.0 33.0
1121.0 188.0
1123.0 156.0
1125.0 465.0
1127.0 288.0
1129.0 39.0
1131.0 31.0
1133.0 73.0
2013.0 3.0
2016.0 0.0
2020.0 185.0
2050.0 22.0
2997.0 0.0
2068.0 0.0
2070.0 2.0
2090.0 44.0
2100.0 0.0
2110.0 3.0
2122.0 20.0
2130.0 1.0
2150.0 6.0
2158.0 4.0
21

20141.0 12.0
20143.0 13.0
20145.0 14.0
20147.0 25.0
20149.0 34.0
20151.0 25.0
20153.0 7.0
20155.0 142.0
20157.0 15.0
20159.0 12.0
20161.0 49.0
20163.0 11.0
20165.0 14.0
20167.0 28.0
20169.0 144.0
20171.0 25.0
20173.0 811.0
20175.0 42.0
20177.0 377.0
20179.0 14.0
20181.0 12.0
20183.0 3.0
20185.0 10.0
20187.0 6.0
20189.0 11.0
20191.0 53.0
20193.0 21.0
20195.0 8.0
20197.0 15.0
20199.0 4.0
20201.0 14.0
20203.0 6.0
20205.0 20.0
20207.0 5.0
20209.0 308.0
21001.0 55.0
21003.0 46.0
21005.0 24.0
21007.0 10.0
21009.0 105.0
21011.0 24.0
21013.0 58.0
21015.0 162.0
21017.0 20.0
21019.0 77.0
21021.0 78.0
21023.0 8.0
21025.0 11.0
21027.0 18.0
21029.0 83.0
21031.0 28.0
21033.0 32.0
21035.0 50.0
21037.0 84.0
21039.0 10.0
21041.0 21.0
21043.0 37.0
21045.0 41.0
21047.0 108.0
21049.0 32.0
21051.0 46.0
21053.0 32.0
21055.0 24.0
21057.0 14.0
21059.0 196.0
21061.0 25.0
21063.0 2.0
21065.0 12.0
21067.0 323.0
21069.0 25.0
21071.0 74.0
21073.0 60.0
21075.0 16.0
21077.0 33.0
21079.0 37.0
21081.0 25.0
21083.0 99.

31075.0 4.0
31077.0 1.0
31079.0 120.0
31081.0 19.0
31083.0 0.0
31085.0 0.0
31087.0 5.0
31089.0 15.0
31091.0 3.0
31093.0 10.0
31095.0 3.0
31097.0 10.0
31099.0 5.0
31101.0 10.0
31103.0 0.0
31105.0 7.0
31107.0 13.0
31109.0 243.0
31111.0 80.0
31113.0 0.0
31115.0 0.0
31119.0 44.0
31117.0 1.0
31121.0 15.0
31123.0 15.0
31125.0 10.0
31127.0 13.0
31129.0 12.0
31131.0 13.0
31133.0 4.0
31135.0 11.0
31137.0 10.0
31139.0 21.0
31141.0 50.0
31143.0 18.0
31145.0 15.0
31147.0 24.0
31149.0 2.0
31151.0 3.0
31153.0 136.0
31155.0 20.0
31157.0 96.0
31159.0 34.0
31161.0 21.0
31163.0 4.0
31165.0 1.0
31167.0 11.0
31169.0 7.0
31171.0 1.0
31173.0 11.0
nan 0.0
31175.0 2.0
31177.0 29.0
31179.0 7.0
31181.0 8.0
31183.0 0.0
31185.0 16.0
32510.0 125.0
32001.0 49.0
32003.0 4721.0
32005.0 37.0
32007.0 60.0
32009.0 0.0
32011.0 0.0
32013.0 14.0
32015.0 9.0
32017.0 3.0
32019.0 69.0
32021.0 8.0
32023.0 111.0
32027.0 20.0
32029.0 3.0
32031.0 692.0
32033.0 8.0
33001.0 101.0
33003.0 13.0
33005.0 35.0
33007.0 45.0
33009.0 29.0


51139.0 61.0
51141.0 46.0
51730.0 89.0
51143.0 85.0
51735.0 18.0
51740.0 205.0
51145.0 17.0
51147.0 36.0
51149.0 33.0
51153.0 514.0
51155.0 69.0
51750.0 28.0
51157.0 2.0
51159.0 12.0
51760.0 281.0
51161.0 148.0
51770.0 200.0
51163.0 48.0
51165.0 114.0
51167.0 40.0
51775.0 50.0
51169.0 61.0
51171.0 134.0
51173.0 96.0
51175.0 57.0
51177.0 125.0
51179.0 83.0
51790.0 70.0
51800.0 193.0
51181.0 9.0
51183.0 23.0
51185.0 76.0
51810.0 421.0
51187.0 61.0
51191.0 115.0
51820.0 39.0
51193.0 27.0
51830.0 14.0
51840.0 48.0
51195.0 105.0
51197.0 81.0
51199.0 59.0
53001.0 24.0
53003.0 31.0
53005.0 235.0
53007.0 74.0
53009.0 16.0
53011.0 303.0
53013.0 6.0
53015.0 102.0
53017.0 23.0
53019.0 8.0
53021.0 123.0
53023.0 6.0
53025.0 133.0
53027.0 82.0
53029.0 31.0
53031.0 4.0
53033.0 1685.0
53035.0 123.0
53037.0 37.0
53039.0 13.0
53041.0 83.0
53043.0 12.0
53045.0 38.0
53047.0 42.0
53049.0 15.0
53051.0 9.0
53053.0 670.0
53055.0 0.0
53057.0 79.0
53059.0 1.0
53061.0 623.0
53063.0 696.0
53065.0 32.0
53067.0 123

In [69]:
K_c.head()

,fips,cases,deaths
0,36047.0,3654,8


In [70]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [71]:
print (K)

{36047.0: 3654}


In [72]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 55.0
1001 240.0
1009 284.0
1013 96.0
1015 520.0
1017 182.0
1031 507.0
1033 239.0
1039 325.0
1043 437.0
1045 526.0
1051 418.0
1055 548.0
1067 138.0
1069 949.0
1071 238.0
1077 433.0
1079 107.0
1083 417.0
1089 1275.0
1095 503.0
1097 5076.0
1103 367.0
1111 114.0
1113 189.0
1115 425.0
1117 1169.0
1121 365.0
2261 123.0
4021 1691.0
5009 386.0
5011 63.0
5033 332.0
5037 127.0
5045 1098.0
5047 104.0
5051 982.0
5053 217.0
5063 298.0
5083 156.0
5085 719.0
5087 93.0
5115 383.0
5117 63.0
5121 76.0
5125 1090.0
5131 775.0
5145 742.0
6007 636.0
6017 571.0
6023 449.0
6027 27.0
6061 1366.0
6079 578.0
6089 431.0
6093 87.0
8019 20.0
8039 44.0
8047 8.0
8051 27.0
8057 1.0
8065 7.0
8067 79.0
8071 47.0
8075 7.0
8077 380.0
8083 28.0
8085 103.0
8093 36.0
8099 16.0
8101 207.0
8103 23.0
8119 46.0
8121 1.0
9005 151.0
9007 196.0
9011 360.0
9015 120.0
10001 222.0
12019 2776.0
12033 2395.0
12089 1472.0
12101 4528.0
12109 2919.0
12113 1210.0
13015 310.0
13039 505.0
13045 492.0
13047 162.0
13057 886.0
13073 674.0


48367 665.0
48373 216.0
48381 583.0
48387 35.0
48395 63.0
48399 31.0
48441 448.0
48459 182.0
48467 282.0
48477 117.0
48503 36.0
49017 8.0
49025 10.0
49055 6.0
50003 26.0
50005 3.0
50011 43.0
50013 4.0
50015 11.0
50017 18.0
50019 23.0
50023 35.0
50025 19.0
50027 27.0
51005 17.0
51007 30.0
51009 66.0
51011 26.0
51015 79.0
51017 2.0
51019 158.0
51023 73.0
51031 110.0
51033 89.0
51035 50.0
51041 641.0
51043 16.0
51045 6.0
51047 53.0
51049 11.0
51053 57.0
51057 24.0
51061 99.0
51063 19.0
51065 15.0
51067 76.0
51069 145.0
51071 33.0
51073 120.0
51075 32.0
51077 11.0
51079 37.0
51083 42.0
51085 266.0
51089 52.0
51093 82.0
51095 104.0
51097 4.0
51099 29.0
51101 28.0
51109 32.0
51113 8.0
51127 69.0
51137 53.0
51139 38.0
51143 78.0
51155 74.0
51165 95.0
51171 46.0
51177 232.0
51181 5.0
51187 59.0
51191 83.0
51193 17.0
51197 90.0
51199 115.0
51550 532.0
51580 11.0
51735 17.0
51800 157.0
51810 812.0
51820 43.0
53003 80.0
53011 880.0
53015 450.0
53023 2.0
53027 163.0
53029 162.0
53035 389.0
53037 1

8055 5.0
8073 8.0
8087 16.0
8109 9.0
8111 0.0
8117 153.0
8125 9.0
9001 1112.0
9009 1239.0
12003 434.0
12007 382.0
12023 825.0
12055 465.0
12057 12506.0
12061 1059.0
12065 99.0
12081 2538.0
12085 1030.0
12087 362.0
12099 9939.0
12115 2310.0
12129 303.0
13007 14.0
13011 52.0
13023 118.0
13029 219.0
13031 378.0
13053 95.0
13075 171.0
13079 40.0
13111 70.0
13119 49.0
13129 154.0
13139 589.0
13191 31.0
13199 83.0
13209 78.0
13213 144.0
13217 389.0
13229 179.0
13247 371.0
13251 73.0
13253 31.0
13263 17.0
13277 398.0
13283 48.0
13319 38.0
13321 81.0
15001 782.0
15007 94.0
16031 33.0
16035 17.0
16037 3.0
16049 23.0
16057 48.0
16067 33.0
16075 28.0
16077 5.0
17069 13.0
17097 843.0
17109 116.0
17143 284.0
17165 104.0
17201 524.0
18097 1968.0
18157 443.0
19047 27.0
19049 66.0
19169 97.0
20019 13.0
20049 0.0
20061 98.0
20069 22.0
20093 7.0
20161 146.0
20189 0.0
21015 345.0
21053 25.0
21069 65.0
21087 59.0
21201 4.0
21217 97.0
21223 33.0
22001 546.0
22003 216.0
22007 295.0
22033 4681.0
22037 270.0


In [73]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 1006
36061 2101
36047 3654 3654
36081 2543
36005 1396


In [74]:
print (covid_cases_county_ny_times)

{1001.0: 240.0, 1003.0: 2676.0, 1005.0: 122.0, 1007.0: 175.0, 1009.0: 284.0, 1011.0: 12.0, 1013.0: 96.0, 1015.0: 520.0, 1017.0: 182.0, 1019.0: 87.0, 1021.0: 219.0, 1023.0: 13.0, 1025.0: 112.0, 1027.0: 57.0, 1029.0: 49.0, 1031.0: 507.0, 1033.0: 239.0, 1035.0: 45.0, 1037.0: 50.0, 1039.0: 325.0, 1041.0: 80.0, 1043.0: 437.0, 1045.0: 526.0, 1047.0: 80.0, 1049.0: 409.0, 1051.0: 418.0, 1053.0: 372.0, 1055.0: 548.0, 1057.0: 43.0, 1059.0: 236.0, 1061.0: 221.0, 1063.0: 37.0, 1065.0: 65.0, 1067.0: 138.0, 1069.0: 949.0, 1071.0: 238.0, 1073.0: 3454.0, 1075.0: 30.0, 1077.0: 433.0, 1079.0: 107.0, 1081.0: 670.0, 1083.0: 417.0, 1085.0: 26.0, 1087.0: 50.0, 1089.0: 1275.0, 1091.0: 65.0, 1093.0: 121.0, 1095.0: 503.0, 1097.0: 5076.0, 1099.0: 151.0, 1101.0: 785.0, 1103.0: 367.0, 1105.0: 17.0, 1107.0: 91.0, 1109.0: 114.0, 1111.0: 114.0, 1113.0: 189.0, 1115.0: 425.0, 1117.0: 1169.0, 1119.0: 34.0, 1121.0: 365.0, 1123.0: 245.0, 1125.0: 783.0, 1127.0: 305.0, 1129.0: 127.0, 1131.0: 40.0, 1133.0: 120.0, 2013.0: 6.

In [75]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [76]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [77]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [78]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [79]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [80]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [81]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [82]:
#ACI
ACI_State = total_state(ACI_total)

In [83]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [84]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [85]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [86]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 775051
ALABAMA 947688
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 283412
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [87]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [88]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [89]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 81.0 38921 208.11387168880552
1001 114.0 55869 204.04875691349406
1009 139.0 57826 240.37630131774637
1013 72.0 19448 370.2180172768408
1015 334.0 113605 294.0011443158312
1017 125.0 33254 375.8946292175377
1031 132.0 52342 252.18753582209314
1033 142.0 55241 257.0554479462718
1039 125.0 37049 337.391022699668
1043 206.0 83768 245.91729538726005
1045 117.0 49172 237.94029122264703
1051 219.0 81209 269.6745434619316
1055 370.0 102268 361.794500723589
1067 45.0 17205 261.5518744551003
1069 293.0 105882 276.7231446327043
1071 117.0 51626 226.62999263936777
1077 254.0 92729 273.91646626190294
1079 100.0 32924 303.7298019681691
1083 158.0 98915 159.73310418035686
1089 533.0 372909 142.93031275726787
1095 236.0 96774 243.86715440097547
1097 865.0 413210 209.33665690568958
1103 291.0 119679 243.15042739327703
1111 44.0 22722 193.64492562274447
1113 43.0 57961 74.18781594520453
1115 252.0 89512 281.52649924032534
1117 259.0 217702 118.96996812156067
1121 188.0 79978 235.0646427767636
226

21091 16.0 8722 183.44416418252695
21093 172.0 110958 155.01360875286144
21097 27.0 18886 142.96304140633274
21101 80.0 45210 176.95200176952002
21103 20.0 16126 124.02331638348008
21107 154.0 44686 344.62695251309134
21117 219.0 166998 131.13929508137824
21123 29.0 14398 201.41686345325738
21125 82.0 60813 134.83959021919654
21141 74.0 27102 273.04257988340345
21145 135.0 65418 206.36522058149134
21149 28.0 9207 304.1164331486912
21151 113.0 92987 121.5223633411122
21157 63.0 31100 202.57234726688105
21161 40.0 17070 234.32923257176333
21163 27.0 28572 94.49811003779924
21179 63.0 46233 136.2663032898579
21185 85.0 66799 127.24741388344135
21191 4.0 14590 27.41603838245373
21199 124.0 64979 190.8308838240047
21209 36.0 57004 63.153462914883164
21211 79.0 49024 161.14556135770235
21213 44.0 18572 236.91578720654746
21215 25.0 19351 129.19228980414448
21227 192.0 132896 144.47387430772935
21229 41.0 12095 338.9830508474576
21239 22.0 26734 82.29221216428519
22015 317.0 127039 249.529671

42089 322.0 170271 189.1103006383941
42095 720.0 305285 235.8451938352687
42099 101.0 46272 218.27455048409405
42103 54.0 55809 96.75858732462505
42105 25.0 16526 151.2767759893501
42107 413.0 141359 292.1639230611422
42109 86.0 40372 213.01892400673734
42111 219.0 73447 298.1741936362275
42115 62.0 40328 153.739337433049
42117 113.0 40591 278.3868345199675
42121 102.0 50668 201.3104918291624
42123 106.0 39191 270.47026102931795
42125 312.0 206865 150.8230005075774
42127 84.0 51361 163.5482175191293
42129 783.0 348899 224.42024769345858
42131 54.0 26794 201.53765768455622
42133 838.0 449058 186.6128651532764
44001 146.0 48479 301.16132758514
44005 43.0 82082 52.38663775249141
44009 172.0 125577 136.9677568344522
45003 226.0 170872 132.26274638325762
45007 537.0 202558 265.10925265849784
45015 209.0 227907 91.70407227509467
45023 80.0 32244 248.10817516437166
45035 228.0 162809 140.04139820280207
45045 1043.0 523542 199.21992886912605
45055 144.0 66551 216.37541133867262
45057 175.0 980

20205 20.0 8525 234.6041055718475
20207 5.0 3138 159.33715742511154
21001 55.0 19202 286.42849703155923
21019 77.0 46718 164.8186994306263
21021 78.0 30060 259.4810379241517
21027 18.0 20477 87.90350148947599
21031 28.0 12879 217.40818386520692
21033 32.0 12747 251.03946026516044
21035 50.0 39001 128.20184097843645
21037 84.0 93584 89.75893315096597
21057 14.0 6614 211.672210462655
21061 25.0 12150 205.761316872428
21067 323.0 323152 99.95296331138289
21077 33.0 8869 372.0825346713271
21081 25.0 25069 99.72475966332922
21085 58.0 26427 219.4725091762213
21105 16.0 4380 365.29680365296804
21111 1348.0 766757 175.80537249741445
21113 75.0 54115 138.59373556315253
21137 61.0 24549 248.48262658356754
21139 25.0 9194 271.9164672612573
21155 39.0 19273 202.35562704301356
21167 54.0 21933 246.20434961017645
21173 30.0 28157 106.54544163085555
21177 70.0 30622 228.5938214355692
21183 55.0 23994 229.22397265983165
21187 13.0 10901 119.25511420970552
21197 13.0 12359 105.18650376244032
21221 15.

22037 121.0 19135 632.3490985105827
22047 102.0 32511 313.7399649349451
22051 944.0 432493 218.26942863815137
22059 38.0 14892 255.17056137523502
22075 27.0 23197 116.39436133982842
22077 75.0 21730 345.1449608835711
22085 64.0 23884 267.9618154412996
22095 142.0 42837 331.48913322594956
22119 116.0 38340 302.55607720396455
22127 48.0 13904 345.22439585730723
23021 7.0 16785 41.70390229371463
24001 212.0 70416 301.0679391047489
24019 65.0 31929 203.57668577155562
24047 106.0 52276 202.7699135358482
25017 3796.0 1611699 235.52784980322008
26001 33.0 10405 317.1552138395002
26033 51.0 37349 136.54984069185252
26035 86.0 30950 277.8675282714055
26049 981.0 405813 241.7369576627659
26069 75.0 25127 298.48370278982765
26095 6.0 6229 96.32364745545031
26129 43.0 20997 204.7911606419965
26153 5.0 8094 61.774153694094394
27031 0.0 5463 0.0
27053 1805.0 1265843 142.59272279421697
27087 9.0 5527 162.83698208793197
27103 45.0 34274 131.29485907685125
28005 44.0 12297 357.81084817435146
28009 25.0

37083 116.0 50010 231.95360927814437
38005 20.0 6832 292.7400468384075
40001 45.0 22194 202.75750202757501
40139 35.0 19983 175.1488765450633
41049 17.0 11603 146.51383262949238
45009 53.0 14066 376.7951087729276
45033 85.0 30479 278.8805407001542
45053 49.0 30073 162.93685365610347
45075 256.0 86175 297.0699158688715
45089 106.0 30368 349.0516332982086
47007 12.0 15064 79.66011683483802
47061 35.0 13427 260.6688016682803
48003 47.0 18705 251.2697139802192
48013 152.0 51153 297.147772369167
48017 19.0 7000 271.42857142857144
48025 75.0 32565 230.30861354214645
48069 32.0 7530 424.96679946879146
48079 15.0 2853 525.7623554153523
48095 8.0 2726 293.470286133529
48101 7.0 1398 500.7153075822604
48103 13.0 4797 271.00271002710025
48113 4180.0 2635516 158.6027176461839
48117 68.0 18546 366.65588267011753
48153 30.0 5712 525.2100840336135
48169 22.0 6229 353.1867073366511
48177 59.0 20837 283.15016557085954
48195 24.0 5399 444.52676421559545
48201 6729.0 4713325 142.76545750611297
48207 28.0

In [90]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(2140.0, 854757, 250.36355361816283)

In [91]:
total_state(Covid_death_capita) 

{'NEW MEXICO': 7795.124966339605,
 'ALABAMA': 19012.705062344423,
 'ALASKA': 903.8522643524606,
 'ARIZONA': 4798.629628973768,
 'ARKANSAS': 18064.911900512237,
 'CALIFORNIA': 6354.7154378210425,
 'COLORADO': 8531.33998214268,
 'CONNECTICUT': 1615.4918845958903,
 'DELAWARE': 606.1285779246226,
 'FLORIDA': 13229.584821335615,
 'GEORGIA': 45550.94261552383,
 'IDAHO': 5318.226454308889,
 'ILLINOIS': 24123.234437099352,
 'INDIANA': 20347.662167365357,
 'IOWA': 25397.716590585773,
 'KANSAS': 28074.574280715846,
 'KENTUCKY': 23059.289791432064,
 'LOUISIANA': 18047.475855459812,
 'MAINE': 954.9712968674353,
 'MARYLAND': 3917.38758653797,
 'MASSACHUSETTS': 3027.0846368619004,
 'MICHIGAN': 16403.47916277224,
 'MINNESOTA': 13254.400912716026,
 'MISSISSIPPI': 25231.511915788866,
 'MISSOURI': 19168.7693301514,
 'MONTANA': 12197.136574365804,
 'NEBRASKA': 14854.953251505694,
 'NEVADA': 2258.2110521767863,
 'NEW HAMPSHIRE': 871.6106609123386,
 'NEW JERSEY': 5840.094926407715,
 'NEW YORK': 9379.779378

In [92]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [93]:
Covid_death_capita[72049]

nan

In [94]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [95]:
print(Medicaid_demand_state)

{'NEW MEXICO': 775051.0, 'ALABAMA': 947688.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 283412.0, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON': 1053931

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [96]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [97]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 775051.0 76321890.0 1000000 10155.028917653899
ALABAMA 947688.0000000001 76321890.0 1000000 12416.988101316674
ALASKA 231145.0 76321890.0 1000000 3028.554455346952
ARIZONA 1839932.0 76321890.0 1000000 24107.526687297704
ARKANSAS 830467.0000000003 76321890.0 1000000 10881.111565764428
CALIFORNIA 11847711.0 76321890.0 1000000 155233.45923430356
COLORADO 1337805.0 76321890.0 1000000 17528.45743206831
CONNECTICUT 874974.0 76321890.0 1000000 11464.260122489104
DELAWARE 239009.0 76321890.0 1000000 3131.591736001297
FLORIDA 3892551.999999999 76321890.0 1000000 51001.77681658564
GEORGIA 1928702.999999999 76321890.0 1000000 25270.639917329077
IDAHO 340742.0000000001 76321890.0 1000000 4464.538286460151
ILLINOIS 2987496.0000000005 76321890.0 1000000 39143.37027031171
INDIANA 1602975.9999999995 76321890.0 1000000 21002.834180338035
IOWA 699741.0 76321890.0 1000000 9168.287106097609
KANSAS 401103.0000000001 76321890.0 1000000 5255.41230700655
KENTUCKY 1416013.0000000002 76321890.0 10000

In [98]:
print (Covid_state)

{'NEW MEXICO': 3998.0, 'ALABAMA': 28475.0, 'ALASKA': 3110.0, 'ARIZONA': 21900.0, 'ARKANSAS': 22869.0, 'CALIFORNIA': 116300.0, 'COLORADO': 10554.0, 'CONNECTICUT': 4623.0, 'DELAWARE': 1260.0, 'FLORIDA': 183891.0, 'GEORGIA': 39977.0, 'IDAHO': 3627.0, 'ILLINOIS': 22882.0, 'INDIANA': 12858.0, 'IOWA': 2159.0, 'KANSAS': 8871.0, 'KENTUCKY': 15469.0, 'LOUISIANA': 50659.0, 'MAINE': 1043.0, 'MARYLAND': 5178.0, 'MASSACHUSETTS': 8077.0, 'MICHIGAN': 9372.0, 'MINNESOTA': 6282.0, 'MISSISSIPPI': 19407.0, 'MISSOURI': 30068.0, 'MONTANA': 2038.0, 'NEBRASKA': 2277.0, 'NEVADA': 13049.0, 'NEW HAMPSHIRE': 1046.0, 'NEW JERSEY': 11897.0, 'NEW YORK': 23533.0, 'NORTH CAROLINA': 30886.0, 'NORTH DAKOTA': 753.0, 'OHIO': 13506.0, 'OKLAHOMA': 8062.0, 'OREGON': 9638.0, 'PENNSYLVANIA': 11157.0, 'RHODE ISLAND': 582.0, 'SOUTH CAROLINA': 21388.0, 'SOUTH DAKOTA': 271.0, 'TENNESSEE': 23843.0, 'TEXAS': 113740.0, 'UTAH': 9681.0, 'VERMONT': 468.0, 'VIRGINIA': 12625.0, 'WASHINGTON': 17525.0, 'WEST VIRGINIA': 2565.0, 'WISCONSIN':

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [99]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [100]:
cartesian_pro_county_state

<gurobi.tuplelist (3220 tuples, 2 values each):
 ( 35039 , NEW MEXICO           )
 ( 1001  , ALABAMA              )
 ( 1009  , ALABAMA              )
 ( 1013  , ALABAMA              )
 ( 1015  , ALABAMA              )
 ( 1017  , ALABAMA              )
 ( 1031  , ALABAMA              )
 ( 1033  , ALABAMA              )
 ( 1039  , ALABAMA              )
 ( 1043  , ALABAMA              )
 ( 1045  , ALABAMA              )
 ( 1051  , ALABAMA              )
 ( 1055  , ALABAMA              )
 ( 1067  , ALABAMA              )
 ( 1069  , ALABAMA              )
 ( 1071  , ALABAMA              )
 ( 1077  , ALABAMA              )
 ( 1079  , ALABAMA              )
 ( 1083  , ALABAMA              )
 ( 1089  , ALABAMA              )
 ( 1095  , ALABAMA              )
 ( 1097  , ALABAMA              )
 ( 1103  , ALABAMA              )
 ( 1111  , ALABAMA              )
 ( 1113  , ALABAMA              )
 ( 1115  , ALABAMA              )
 ( 1117  , ALABAMA              )
 ( 1121  , ALABAMA              )


In [101]:
Total_covid_death_cap


{'NEW MEXICO': 7795.124966339605,
 'ALABAMA': 19012.705062344423,
 'ALASKA': 903.8522643524606,
 'ARIZONA': 4798.629628973768,
 'ARKANSAS': 18064.911900512237,
 'CALIFORNIA': 6354.7154378210425,
 'COLORADO': 8531.33998214268,
 'CONNECTICUT': 1615.4918845958903,
 'DELAWARE': 606.1285779246226,
 'FLORIDA': 13229.584821335615,
 'GEORGIA': 45550.94261552383,
 'IDAHO': 5318.226454308889,
 'ILLINOIS': 24123.234437099352,
 'INDIANA': 20347.662167365357,
 'IOWA': 25397.716590585773,
 'KANSAS': 28074.574280715846,
 'KENTUCKY': 23059.289791432064,
 'LOUISIANA': 18047.475855459812,
 'MAINE': 954.9712968674353,
 'MARYLAND': 3917.38758653797,
 'MASSACHUSETTS': 3027.0846368619004,
 'MICHIGAN': 16403.47916277224,
 'MINNESOTA': 13254.400912716026,
 'MISSISSIPPI': 25231.511915788866,
 'MISSOURI': 19168.7693301514,
 'MONTANA': 12197.136574365804,
 'NEBRASKA': 14854.953251505694,
 'NEVADA': 2258.2110521767863,
 'NEW HAMPSHIRE': 871.6106609123386,
 'NEW JERSEY': 5840.094926407715,
 'NEW YORK': 9379.779378

# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [102]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [103]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [104]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [105]:
#print (Medicaid_demand)

In [106]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [107]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [108]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [109]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)